In [307]:
import pandas as pd
from tqdm.notebook import tqdm
import csv
import requests
import statsmodels.formula.api as sm
import numpy as np

In [308]:
# create the date csv  - DONE

In [309]:
# system load pathname and retrieval?? - SKIP

In [310]:
# add the new lagged week variables || remove the old one - DONE

In [311]:
# add lag48 - DONE

In [312]:
# mark rows with bad data + delete those rows at end - SKIP

In [313]:
## Inputs

year = 2021

#Choose the node. For a full list, see https://www.iso-ne.com/markets-operations/settlements/pricing-node-tables/
node = ['LD.NEW_HAVN46']

In [314]:
## Iterable including all dates to be retrieved

year_start = str(year) + '-01-01'
year_end = str(year + 1) + '-01-01'
leap_check = pd.Timestamp(year, 1, 1)

#create a dataframe including every day of the year
df_date_time = pd.DataFrame(
        {'Days': pd.date_range(year_start, year_end, freq='1D', inclusive='left')}
     )

#convert into yearmonthday format, e.g. Jan., 1st, 2020 = 20200101
df_date_time['year'] = pd.DatetimeIndex(df_date_time['Days']).year

df_date_time['month'] = pd.DatetimeIndex(df_date_time['Days']).month
df_date_time["month"] = df_date_time.month.map("{:02}".format)

df_date_time['day'] = pd.DatetimeIndex(df_date_time['Days']).day
df_date_time['day'] = df_date_time.day.map("{:02}".format)

df_date_time['date_time'] = df_date_time['year'].astype(str) + df_date_time['month'] + df_date_time['day']

df_date_time = df_date_time.drop(['Days', 'month', 'day', 'year'], axis=1)

if leap_check.is_leap_year:
    df_date_time = df_date_time.drop(365)

In [315]:
#retrieve and clean first day, and define initial dataframe
url = "https://www.iso-ne.com/static-transform/csv/histRpts/rt-lmp/lmp_rt_final_" + df_date_time['date_time'][0] + ".csv"
main_df = pd.read_csv(url, skiprows=4, usecols=range(1,10))

#Remove other nodes
main_df = main_df[main_df['Location Name'].isin(node)]
main_df = main_df.reset_index(drop=True)

#Create replacement dataframe (for when data doesn't exist)
replacement_df = main_df.iloc[0:24]

In [316]:
#retrieve remaining days and concat with main_df

#drop retrieved day
first_day = df_date_time['date_time'][0]
df_date_time = df_date_time.drop(0)

#retrieve data, clean and concat
for x in tqdm(df_date_time.index):
#     print(df_date_time['date_time'][x])
    
    url_iter = "https://www.iso-ne.com/static-transform/csv/histRpts/rt-lmp/lmp_rt_final_" + str(df_date_time['date_time'][x]) + ".csv"

    request = requests.get(url_iter)
    
    if request.status_code == 200: 
        temp_df = pd.read_csv(url_iter, skiprows=4, usecols=range(1,10))

        #Remove other nodes
        temp_df = temp_df[temp_df['Location Name'].isin(node)]

        main_df = pd.concat([main_df, temp_df], ignore_index=True)
    else: 
        main_df = pd.concat([main_df, replacement_df], ignore_index=True)

  0%|          | 0/364 [00:00<?, ?it/s]

In [317]:
#Retrieve and clean system load and add to main_df ## NOTE: due to NEISO server error, system load files have to be downloaded manually.
pathname = "rt_hourlysysload_" + str(first_day) + "_" + str(df_date_time['date_time'][364]) + ".csv"

system_load_df = pd.read_csv(pathname, skiprows=4, usecols=range(1,4))
system_load_df = system_load_df.drop([0, 8761])
system_load_df = system_load_df.reset_index()

main_df['Total_Load'] = system_load_df['Total Load']

#Rename certain columns to remove spaces (due to naming issues with libs)
main_df.rename(columns = {'Locational Marginal Price':'LMP', 'Hour Ending':'Hour_Ending'}, inplace = True)

In [318]:
## Capture lagged price variables

In [319]:
# capture individual lagged prices, last 24hrs
for x in range(24,49):
    column_name = str("lag" + str(x))
    main_df[column_name] = main_df['LMP'].shift(x)

In [320]:
## capture daily average

main_df.to_csv("temporary.csv")
data = pd.read_csv("temporary.csv")

# remove daylight savings
data = data[data['Hour_Ending'] != "02X"]


data['Hour_Ending'] = data['Hour_Ending'].astype(int) - 1
data['Date'] = pd.to_datetime(data['Date']) + pd.to_timedelta(data['Hour_Ending'], unit='h')
data = data.drop(['Hour_Ending'], axis=1)

davg_data = data.resample('1D', on='Date').mean()

davg_data = davg_data[['LMP', 'Total_Load']].drop(['Total_Load'], axis=1)
davg_data['Date_col'] = davg_data.index

data = pd.read_csv("temporary.csv", index_col=0)
data['Date'] = pd.to_datetime(data['Date'])

merged_data = data.merge(davg_data, left_on='Date', right_on='Date_col')
merged_data['LMP_y'] = merged_data['LMP_y'].shift(periods=48)

main_df = merged_data

In [321]:
# capture calendar variables
main_df['Date']= pd.to_datetime(main_df['Date'])
main_df['month'] = main_df['Date'].dt.month
main_df['day'] = main_df['Date'].dt.day

In [322]:
# retrieve & add NEISO solar production
solar_df = pd.read_excel('https://www.iso-ne.com/static-assets/documents/2021/04/hourly_solar_gen_' + str(year) + '.xlsx', sheet_name='HourlyData')
solar_df = solar_df.drop(['year', 'local_day', 'LOCAL_HOUR_END', '_FREQ_'], axis=1)
main_df['tot_solar_mwh'] = solar_df['tot_solar_mwh']

# retrieve & add NEISO wind production ## NOTE:
wind_df = pd.read_excel('https://www.iso-ne.com/static-assets/documents/2021/04/hourly_wind_gen_' + str(year) + '.xlsx', sheet_name='HourlyData')
wind_df = wind_df.drop(['year', 'local_day', 'local_hour_end', '_FREQ_'], axis=1)
main_df['tot_wind_mwh'] = wind_df['tot_wind_mwh']

In [323]:
# drop rows with empty lagged variables
main_df = main_df.iloc[48:]

# set null solar & wind values to zero.
main_df['tot_solar_mwh'] = main_df['tot_solar_mwh'].fillna(0)
main_df['tot_wind_mwh'] = main_df['tot_wind_mwh'].fillna(0)

# final cleaning
main_df = main_df.drop(['Date_col'], axis=1)
main_df = main_df.rename(columns={"LMP_x":"LMP", "LMP_y":"last_day_LMP"})

/var/folders/jn/q32kcbm50yj0h2rkg7kx1myw0000gn/T/ipykernel_51737/2272136848.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['tot_solar_mwh'] = main_df['tot_solar_mwh'].fillna(0)
/var/folders/jn/q32kcbm50yj0h2rkg7kx1myw0000gn/T/ipykernel_51737/2272136848.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['tot_wind_mwh'] = main_df['tot_wind_mwh'].fillna(0)


In [324]:
main_df

,Date,Hour_Ending,Location ID,Location Name,Location Type,LMP,Energy Component,Congestion Component,Marginal Loss Component,Total_Load,...,lag44,lag45,lag46,lag47,lag48,last_day_LMP,month,day,tot_solar_mwh,tot_wind_mwh
48,2021-01-03,01,4456,LD.NEW_HAVN46,NETWORK NODE,22.97,23.02,0.0,-0.05,12076.09,...,25.08,25.00,24.71,44.44,26.47,30.030417,1,3,0.0,432.135
49,2021-01-03,02,4456,LD.NEW_HAVN46,NETWORK NODE,24.92,24.89,0.0,0.03,11971.59,...,24.69,25.08,25.00,24.71,44.44,30.030417,1,3,0.0,355.170
50,2021-01-03,03,4456,LD.NEW_HAVN46,NETWORK NODE,24.96,24.90,0.0,0.06,11793.21,...,37.45,24.69,25.08,25.00,24.71,30.030417,1,3,0.0,355.055
51,2021-01-03,04,4456,LD.NEW_HAVN46,NETWORK NODE,25.06,24.99,0.0,0.07,11746.35,...,31.61,37.45,24.69,25.08,25.00,30.030417,1,3,0.0,352.174
52,2021-01-03,05,4456,LD.NEW_HAVN46,NETWORK NODE,25.07,25.02,0.0,0.05,11833.52,...,23.94,31.61,37.45,24.69,25.08,30.030417,1,3,0.0,301.128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2021-12-31,20,4456,LD.NEW_HAVN46,NETWORK NODE,33.73,33.93,0.0,-0.20,14183.77,...,37.40,38.10,37.29,37.23,38.67,36.040417,12,31,0.0,121.746
8756,2021-12-31,21,4456,LD.NEW_HAVN46,NETWORK NODE,31.14,31.28,0.0,-0.14,13515.28,...,34.95,37.40,38.10,37.29,37.23,36.040417,12,31,0.0,152.635
8757,2021-12-31,22,4456,LD.NEW_HAVN46,NETWORK NODE,31.18,31.38,0.0,-0.20,12888.40,...,31.61,34.95,37.40,38.10,37.29,36.040417,12,31,0.0,119.166
8758,2021-12-31,23,4456,LD.NEW_HAVN46,NETWORK NODE,31.11,31.28,0.0,-0.17,12217.09,...,30.74,31.61,34.95,37.40,38.10,36.040417,12,31,0.0,107.843


In [325]:
main_df.to_csv(str(year) + '_full_data.csv')